In [2]:
# Check if python is 3.9.5
import arrow
import numpy as np
import os
import sys

from __init__ import PCAT

print(sys.version)
%load_ext autoreload
%autoreload 2
%pprint

3.9.5 (default, May 18 2021, 12:31:01) 
[Clang 10.0.0 ]
Pretty printing has been turned OFF


# Classifying the diversity of Land Surface (Sub)Models

Here, we try to explore and rank LSM for the help on deciding, which model to take. The criteria for it are listed in the notebook [02_submodel_criteria](./02_submodel_criteria.ipynb).

## Structure

For each model we should have a (very) short paragraph that describes the submodel. In addition rankings for all 7 criteria, as well as some reasoning for it should be provided. For each submodel we can then define a python dictionary and cncatenate all of them finally to save it as `csv` or `yaml` file.

In [4]:
all_models_ranked = []

In [5]:
example_model = {}
example_model["key"] = "XX.XX"
example_model["model"] = "ex_mod"
example_model["submodel"] = "example"
example_model["date_summarized"] = arrow.now().format(fmt= 'YYYY-MM-DD HH:mm')
example_model["link"] = "reasoning"
example_model["references"] = "link"

example_model["c1"] = (1, "reasoning")
example_model["c2"] = (2, "reasoning")
example_model["c3"] = (3, "reasoning")
example_model["c4"] = (3, "reasoning")
example_model["c5"] = (2, "reasoning")
example_model["c6"] = (1, "reasoning")
example_model["c7"] = (1, "reasoning")
example_model["c8"] = (1, "reasoning")


all_models_ranked.append(example_model)

---
---
## Models

Let $c_n \in (c_k)_{k \in \mathbb{N}}$ always be predefined constants and us start.

---
### CLM Submodels

We start with looking at the submodels of the CLM. Its [Technical Note](https://escomp.github.io/ctsm-docs/versions/master/html/tech_note/index.html) lists 30 submodules/submodels. These should be analyzed. To avoid a bias of ranking the models in a given order, we fetch them randomly:

<hr style="border-style: dashed;">

#### 22. External Nitrogen Cycle

The external nitrogen cycle can be thought of as processes around nitrogen that happen oustide of organisms, especially plants. They are important to describe long term ecosystem dynamics of nitrogen stocks.

The model parameters are:


<div style="width:100px">parameter</div> | <div style="width:100px">unit</div>  | <div style="width:300px">equation (relation)</div> | <div style="width:500px">description</div>  |
|:---:|:---:|:---:|:---|
$NF_{ndep\_sminn}$ | $g\ m^{-2}\ s^{-1}$ | - | total deposition of mineral nitrogen to land surface (NO$_{x}$, NH$_{y}$)
$CF_{ann\_ET}$ | $mm \ yr^{-1}$ | - | evapotranspiration rate
$NF_{nfix\_sminn}$ | $g\ m^{-2}\ s^{-1}$ | $c_1 (c_2 + CF_{ann\_ET}) / (c_3)$ (linear) | biological asymbiotic nitrogen fixation rate (BNF)
$k_{nitr}$ | $day^{-1}$ | - | maximum nitrification rate (set to 0.1)
$f_{nitr,p}$ | $day^{-1}$ | $[NH_{4}]\ k_{nitr}\ f(T)\ f(H_{2}0)\ f(pH)$ (linear?!) | potential nitrification rate
$frac_{anox}$ | - | $\exp{\text(multiple\ params)}$ (exponential) | anoxic fraction
$f_{denitr,p}$ | $day^{-1}$ | $min(f_{denitr,p,carbon\_lim}, f_{denitr,p,no3\_lim})\ frac_{anox}$ (linear?!) | potential denitrification rate (only anoxic soils)
$k_{1}$ | - | $max(c_1, c_2-c_3\ d_g)$ (linear) | function of gas diffusion
$f_{WFPS}$ | - | $max(c_1, c_2-c_3\ WFPS)$ (linear) | function of water filled pore space
$P_{NO_3:CO_2}$ | - | - |  CO$_2$ production/NO$_{3}$ concentration ratio
$P_{N_2:N_2O}$ | - | $max(0.16\ k_1, k_1\ \exp(-0.8\ P_{NO_3:CO_2}))\ f_{WPFS}$ (exponential) | nitrification/denitrification ratio
$WS_{tot\_soil}$ | $kg\ m^{2}$ | - | total mass of soil water over column
$NS_{sminn}$ | $g\ m^{-2} s^{-1}$ | - | soulable fraction of anorganic nitrogen in water
$DIN$ | $g\ kg^{-1}$ | $\frac{NS_{sminn}\ s\ f}{WS_{tot\_soil}} $ (linear?!) | dissolved anorganic nitrogen in water solution 
$Q_{dis}$ | $kg\ m^{-2}\ s^{-1}$ | - | hydrologic discharge from soil column to streamflow
$NF_{leached}$ | $g\ m^{-2} s^{-1}$ | $min(DIN\ Q_{dis}, \frac{NS_{sminn}\ s\ f}{\Delta t} )$ (linear?!) | leaching loss of nitrogen

Processes:
- Deposition of atmospheric nitrogen
- Biological nitrogen fixation
- Nitrification and denitrification
- Leaching of nitrogen

Bottlenecks:
- Steady state approximations (e.g. constants in BNF, maximum denitrification rate, nitrification flux)
- Binning of seperate processes (e.g. deposition of different atmospheric nitrogen compounds to one variable)
- Simplified fluxes (e.g. atmospheric N deposition and BNF contribute solely to NH$_4$ pool)
- Proxying processes by others (e.g. BNF by evapotranspiration)

Connected submodels:
- 2.18. Fixation and Uptake of Nitrogen
- 2.19. Carbon and Nitrogen allocation
- 2.24 Fire
...

Concluding comment:

*We should keep in mind that N and C cicle are higly dependent of each other.*

In [7]:
model = {}
example_model["key"] = "2.22"
example_model["model"] = "clm"
example_model["submodel"] = "n_cycle"
# model["date_summarized"] = arrow.now().format(fmt= 'YYYY-MM-DD HH:mm')
model["date_summarized"] = '2022-04-25 17:01'
model["link"] = "https://escomp.github.io/ctsm-docs/versions/master/html/tech_note/External_Nitrogen_Cycle/CLM50_Tech_Note_External_Nitrogen_Cycle.html#leaching-losses-of-nitrogen"
model["useful_references"] = ["kovenEffectVerticallyResolved2013", "thomasNitrogenLimitationLand2015"]

model["c1"] = (3, "Directly linked to biogeochemical flows")
model["c2"] = (2, "Nitrogen availability is an important factor for forests")
model["c3"] = (1, "Not directly linked to climate change issues")
model["c4"] = (2, "Many equations but still not too complex")
model["c5"] = (3, "Involves a lot of parameters, several linked exponentially")
model["c6"] = (3, "Well referenced and parameters nicely defined")
model["c7"] = (2, "Depends on several other nitrogen and water submodels, but builds a good independent core.")
model["c8"] = (1, "No direct connection to biotic evolution")


all_models_ranked.append(model)